In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
import pandas as pd
import time
import matplotlib.pyplot as plt
import plotly.express as px

from pathlib import Path
ROOT_DIR=Path(os.getcwd()).parent.parent
if not str(ROOT_DIR) in sys.path:
    sys.path.append(str(ROOT_DIR))
for p in sys.path:
    print(p)

In [ ]:
from src.data.equity_data.tradingview import BigA
biga = BigA()

In [ ]:
stocks, etfs, foreign_etf = biga.load_data_cache_from_csv(dt='2023-12-29')

In [ ]:
#BigA.top_sharp_ratios(stocks)

In [ ]:
#BigA.alpha_scatter(stocks)

In [ ]:
#BigA.alpha_histogram(stocks)
stocks['name_cn'].shape

In [ ]:
import efinance as ef

In [ ]:
import datetime
today = datetime.datetime.today().strftime('%Y-%m-%d')

import sqlite3
conn = sqlite3.connect(ROOT_DIR / 'database' / 'China_A_Share.db')

# 从数据库中读取已经下载的股票代码
query = f"""
SELECT DISTINCT 股票代码 FROM history_quote
"""
df = pd.read_sql(query, conn)
cached_tickers=df['股票代码'].values.tolist()

stocks_to_download = stocks[~stocks.tic.isin(cached_tickers)]

from tqdm import tqdm
with tqdm(total=stocks_to_download.shape[0], desc="下载历史行情") as pbar:
    for i, row in stocks_to_download.iterrows():
        df = ef.stock.get_quote_history(stock_codes=row.tic)
        df.to_sql('history_quote', conn, if_exists='append', index=False)
        pbar.update(1)